### 230801 RNAseq, oxymax, and metabolomics
- going to repeat the VO2 correction for the RNAseq data
- going to try and correlate the RNAseq and metabolomics data, with and without VO2 correction

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from statsmodels.formula.api import ols
import seaborn as sns

below will be the directories for:
- metabolon data
    - this is, for now, vehicle and 2HB data of feature-wise normalized and scaled metabolite values
    - FWN values subject to VO2 adjustment via OLS with avg VO2 and avg dVO2 during the 18 m/min as predictors
    - serum, RG, and WG values
- RNAseq data
    - want the regularized log transformed count data, which normalizes the data to equalize the reads across samples, and log-scales the data according to the count data set dispersion regression.

In [2]:
path = '/Users/brennanwadsworth/Library/CloudStorage/OneDrive-KI.SE/Documents/BJW Experiments/BJW0037_Metabolon 2HB and exercise/RNASeq/metabolon_RNAseq_comparison/'
for root, dir, file in os.walk(path):
    file_list = file

for item in file_list:
    print(item)

.DS_Store
muscle_adjusted_values_WG.csv
muscle_fwn_RG.csv
muscle_RNAseq_WG.csv
specimen_2.csv
muscle_RNAseq_RG_3.csv
muscle_RNAseq_RG_2.csv
serum_adjusted_values.csv
muscle_adjusted_values_RG.csv
muscle_RNAseq_WG_3.csv
muscle_fwn_WG.csv
muscle_RNAseq_RG.csv
muscle_RNAseq_WG_2.csv
serum_fwn.csv
WG_seq_adjusted_values.csv
RG_seq_adjusted_values.csv


#### RNAseq VO2 adjustment
- need a df with the RNAseq data, VO2 data, and group information
    - read all data into dfs
    - transpose RNAseq df
    - append the group and VO2 columns to the RNAseq df

In [3]:
# met tables need to have the MouseID col set to the index
# RNAseq tables need to be transposed, this has the effect of the column named 'gene' lining up with the MouseId, set this to index
RG_adj = pd.read_csv(f"{path}muscle_adjusted_values_RG.csv").set_index('MouseID').drop("VS100",axis=0)
RG_fwn = pd.read_csv(f"{path}muscle_fwn_RG.csv").set_index('MouseID').drop("VS100",axis=0).drop(['max_dVO2','max_VO2','AUC'],axis=1)
RG_seq = pd.read_csv(f"{path}muscle_RNAseq_RG_3.csv").set_index('gene').transpose()
WG_adj = pd.read_csv(f"{path}muscle_adjusted_values_WG.csv").set_index('MouseID')
WG_fwn = pd.read_csv(f"{path}muscle_fwn_WG.csv").set_index('MouseID').drop(['max_dVO2','max_VO2','AUC'],axis=1)
WG_seq = pd.read_csv(f"{path}muscle_RNAseq_WG_3.csv").set_index('gene').transpose()
ser_adj = pd.read_csv(f"{path}serum_adjusted_values.csv").set_index('MouseID')
ser_fwn = pd.read_csv(f"{path}serum_fwn.csv").set_index('MouseID').drop("VS100",axis=0).drop(['max_dVO2','max_VO2','AUC'],axis=1)

print([RG_adj.shape, RG_fwn.shape, RG_seq.shape, WG_adj.shape, WG_fwn.shape, WG_seq.shape, ser_adj.shape, ser_fwn.shape])

[(23, 532), (23, 532), (35, 13022), (24, 530), (24, 530), (36, 13022), (24, 826), (23, 826)]


In [4]:
RG_adj.oxoadipate.head()

MouseID
HR40   -1.467669
HR70    0.812456
HR92    0.341596
HR52    0.696459
HR81    0.189109
Name: oxoadipate, dtype: float64

In [5]:
RG_seq.A1bg.head()
# removed all genes with numbers at the start of their name.

VR11    13.912456
VS23    15.714834
GR30    15.071795
GS32    19.323242
HR40    16.961272
Name: A1bg, dtype: float64

In [6]:
import re
# goal is to rename the columns in RNA_seq file so that they do not have special characters
# use dataframe.rename function
# need to produce a dictionary that tells the program what to change each col to 
# This would actually be useful to have save so that we can change the names back later into the gene symbol compatable with KEGG etc.

def fix_gene_names (df):
#    pattern = r'[0-9]'
    name = {}
    for gene in df.columns:
        new_str = re.sub(r'[\W_]', '', gene)
#        new_str = re.sub(pattern,"",new_str)
        name[gene] = new_str
    return name

rna_seq_names = fix_gene_names(RG_seq)
RG_seq.rename(columns=rna_seq_names, inplace=True)
WG_seq.rename(columns=rna_seq_names, inplace=True)

In [7]:
RG_info = RG_fwn.iloc[:,-5:] # all rows, final 5 columns
WG_info = WG_fwn.iloc[:,-5:] # all rows, final 5 columns
RG_info.head()

,Group,Tx,Ex,avg_dVO2,avg_VO2
MouseID,,,,,
VR11,VR,V,R,0.292980,0.177744
VR60,VR,V,R,0.595730,0.225418
VR83,VR,V,R,-0.535343,0.996155
VR42,VR,V,R,-0.223137,-0.587017
VR72,VR,V,R,0.582898,1.079038


In [8]:
RG_seq = RG_seq.copy().loc[RG_info.index]
WG_seq = WG_seq.copy().loc[WG_info.index]
RG_seq = RG_seq.join(RG_info)
WG_seq = WG_seq.join(WG_info)

print(RG_seq.shape,WG_seq.shape)

(23, 13027) (24, 13027)


In [9]:
RG_seq_adj = pd.DataFrame(columns=RG_seq.columns[:-5]) # frame for holding adjusted values
covars=['avg_dVO2','avg_VO2']
grouped = RG_seq.groupby('Tx') # organize data by Tx

for gene in RG_seq.columns[:-5]:
    # print(gene) # for tracking progress in loop
    model = ols(formula=f'{gene} ~ {covars[0]} + {covars[1]}', data=grouped.get_group('V')).fit()
    # C(Group) tells the model to expect a categorical variable

    hb_adj = grouped.get_group('H')[gene] - model.predict(grouped.get_group('H')) + model.params[0]
    veh_adj = model.resid + model.params[0]

    RG_seq_adj[gene] = pd.concat([hb_adj,veh_adj],axis=0)

RG_seq_adj = RG_seq_adj.join(RG_info)
RG_seq_adj.to_csv('RG_seq_adjusted_values.csv')


In [10]:
WG_seq_adj = pd.DataFrame(columns=WG_seq.columns[:-5]) # frame for holding adjusted values
covars=['avg_dVO2','avg_VO2']
grouped = WG_seq.groupby('Tx') # organize data by Tx

for gene in WG_seq.columns[:-5]:
    # print(gene) # for tracking progress in loop
    model = ols(formula=f'{gene} ~ {covars[0]} + {covars[1]}', data=grouped.get_group('V')).fit()
    # C(Group) tells the model to expect a categorical variable

    hb_adj = grouped.get_group('H')[gene] - model.predict(grouped.get_group('H')) + model.params[0]
    veh_adj = model.resid + model.params[0]

    WG_seq_adj[gene] = pd.concat([hb_adj,veh_adj],axis=0)

WG_seq_adj = WG_seq_adj.join(WG_info)
WG_seq_adj.to_csv('WG_seq_adjusted_values.csv')

In [11]:
RG_seq_adj.head()

,A130010J15Rik,A1bg,A230009B12Rik,A230056J06Rik,A230072C01Rik,A330009N23Rik,A330023F24Rik,A330041J22Rik,A330074K22Rik,A430005L14Rik,...,Zxdc,Zyg11b,Zyx,Zzef1,Zzz3,Group,Tx,Ex,avg_dVO2,avg_VO2
MouseID,,,,,,,,,,,,,,,,,,,,,
HR40,100.335158,15.999794,9.963374,1235.778921,43.850132,22.654843,62.227333,22.714789,80.037643,486.509409,...,175.782605,1208.334726,39.611826,266.568035,331.647275,HR,H,R,1.622066,-0.585547
HR70,100.231160,10.508283,24.431187,1134.640900,36.773066,20.827650,55.100144,26.411284,91.648578,607.447231,...,223.525883,1252.414541,270.130475,287.028653,260.644720,HR,H,R,1.450905,0.939831
HR92,150.992718,27.240973,13.498595,720.439231,67.894596,26.958223,58.450226,47.939038,32.535427,1189.025745,...,135.377457,921.653705,433.083687,210.260724,309.345013,HR,H,R,0.556003,-0.499269
HR52,116.760921,11.406998,10.369015,1369.842026,32.865368,21.949431,103.304817,22.487331,80.596439,323.782416,...,334.034185,1782.811570,488.301841,417.300880,411.286024,HR,H,R,0.496940,0.196349
HR81,100.470459,12.224968,14.441181,1304.187948,36.576294,17.377701,75.480510,29.186421,73.568750,543.218919,...,282.515457,1532.858723,281.630633,315.811320,292.358248,HR,H,R,0.761212,0.854730


In [197]:
RG_df = RG_seq_adj.drop(['Group','Tx','Ex', 'avg_dVO2', 'avg_VO2'],axis=1).join(RG_adj.drop(['Group','Tx','Ex', 'avg_dVO2', 'avg_VO2'],axis=1))
RG_df.head()

,A130010J15Rik,A1bg,A230009B12Rik,A230056J06Rik,A230072C01Rik,A330009N23Rik,A330023F24Rik,A330041J22Rik,A330074K22Rik,A430005L14Rik,...,hydroxypalmitoylsphingomyelindOH,hydroxyNNNtrimethyllysine,hydroxydecanoylcarnitine,succinoyltaurine,hexanoyltaurine,Nacetylaminoadipate,ditertbutylphenol,NNdimethylpropro,oxindolylalanine,oleylGPCO
MouseID,,,,,,,,,,,,,,,,,,,,,
HR40,6.752415,4.207428,3.526195,10.223330,5.342715,4.377695,6.239402,4.691817,6.046739,8.971924,...,-2.713307,2.902504,3.497966,-1.146348,-1.230095,-1.129802,-2.074813,-0.420198,-2.253050,0.949912
HR70,6.752665,4.061303,3.900203,10.148209,5.221494,4.346190,6.135492,4.784510,6.217978,9.197804,...,1.366213,1.469657,1.775979,-0.561337,-1.104361,-0.734704,-0.781902,0.299352,-1.856883,-0.588613
HR92,7.069322,4.481420,3.616102,9.747524,5.713990,4.504608,6.198355,5.188141,5.343345,9.870979,...,-1.132891,0.333584,1.428994,0.562333,0.303499,0.525386,-0.173917,-0.767051,-1.005226,0.647059
HR52,6.868872,4.103060,3.525992,10.308941,5.162718,4.390634,6.584357,4.711890,6.087094,8.655739,...,-0.392374,0.412405,1.555703,-0.737090,-0.302112,0.749926,-0.775116,0.393152,-0.293635,-0.035675
HR81,6.758159,4.129208,3.649669,10.263256,5.234578,4.271676,6.345743,4.852893,6.035895,9.106773,...,-2.196238,-0.844257,-0.077946,-1.270169,-1.126717,-0.509758,0.061403,-0.051534,-1.379212,-0.882308
